In [1]:
import numpy as np # dizisel işlemler ve matematiksel hesaplamalar için 
import pandas as pd #  veri seti için
import matplotlib.pyplot as plt # veri görselleştirmeleri için 
from sklearn.preprocessing import MinMaxScaler # verilerin normalizasyon süreci için
from tensorflow.keras.models import Sequential # veri katmanlarını oluşturacagımız süreçte kullanacaz bunu da
from tensorflow.keras.layers import LSTM,Dense,Dropout # lstm katmanları ve nöron yapıları için kullanacağımız kütüphane
import tensorflow as tf
# Eski oturumları ve değişkenleri RAM'den tamamen siler
tf.keras.backend.clear_session() 

# TensorFlow'u "satır satır çalışmaya" (Eager Mode) zorlar
tf.config.run_functions_eagerly(True)



In [2]:

dosya_yolu='/Users/sametkarayel/Desktop/live_tracking/veri/btcusd_1-min_data.csv'

df=pd.read_csv(dosya_yolu) # verimizi dataframe e çevirdik 

df=df.tail(250000).reset_index(drop=True)

df=df[['Open','High','Low','Close','Volume']].copy() # sadece bu 5 sütunu modelin eğitim sürecinde faydalı diye bunları tek alalım



In [3]:
df.isnull().sum() # veride boş değer varmı kısaca onu kontrol ettik 
df.dropna()

,Open,High,Low,Close,Volume
0,114723.0,114735.0,114722.0,114735.0,0.011070
1,114734.0,114735.0,114730.0,114735.0,0.675345
2,114725.0,114742.0,114721.0,114742.0,0.236531
3,114746.0,114795.0,114746.0,114795.0,0.029711
4,114794.0,114794.0,114735.0,114735.0,1.016532
...,...,...,...,...,...
249995,68866.0,68880.0,68862.0,68862.0,0.108650
249996,68868.0,68868.0,68852.0,68852.0,0.186825
249997,68852.0,68860.0,68812.0,68812.0,2.609525
249998,68812.0,68817.0,68797.0,68803.0,0.360523


In [4]:
scaler = MinMaxScaler(feature_range=(0,1)) # normalizasyonumuzu uyguluyoruz parametre değerleri olarak hangi değer aralıgına dönüştürülmesini istiyorsak gireceğiz
# örnek biz şuanda 0 ile 1 arasında olmasını istiyoruz
scaled_data = scaler.fit_transform(df) # tüm df ye birden bunu uygulayıp uyguladıgımız haline de scaled_data(ölçeklendirilmiş veri adını atıyoruz)

In [5]:
X,y = [], [] # biz modelin api dan gelen verileri değerlendirip tahminde bulunmasını amaçlıyoruz bunun için geçmiş 60 dk lik veriyi değerlendirip gelecek 1 dk için bir tahminde bulunmasını isteyeceğiz 
# bu yapılan işlemi pencerelere bölme(Windowing) olarak adlandırıyoruz
#X geçmişten alınan 60 dk lik veri 
#y ise tahminde bulunulacak gelecek 1 dk veri 
window_size = 60 # bu parametre modelin son 60 satıra bakarak bir sonraki değeri tahmin etmesini sağlayacak

In [6]:
for i in range(window_size, len(scaled_data)):# döngü 60. indeksten veri setinin sonuna kadar döner ;60 la başlıyor çünkü tahmin için 60 veri gerekiyor ve ilk 60 veriden önce 60 verilik pencere oluşturacak kadar veri yok 
    X.append(scaled_data[i-window_size:i]) #bu satırda girdi setini oluşturacak i o anki son adımsa, i-60 adımına kadar alacak yani kendi(i) ve önceki 59 adımı alacak
    y.append(scaled_data[i,3])#burya da modelin öğrenmesi gereken hedef(target) sütun değerleri eklenecek 

In [7]:
X, y = np.array(X), np.array(y) # X ve y listelerini numpy dizilerine çeviriyoruz modelde kullanmak için

model=Sequential() # boş modeli inşa ediyoruz ( sequential = katmanlı model olarak düşünebiliriz )

In [8]:
model.add(LSTM(units=50, return_sequences=True,input_shape=(X.shape[1],X.shape[2])))
#ilk katmanı inşa ediyoruz bu katmanda 50  nöronlu bir yapı kuruyoruz
# sonradan bir katman daha ekleyecegimiz için eturn_sequences parametresini True  giriyoruz
# input_shape=(X.shape[1],X.shape[2]) burda girdigimiz parametreler ilki satır yani X window size değeridir ikinci ise sütun yani feature değeri ki bu başta ekledigimiz gibi 5
model.add(Dropout(0.2)) 
# dropout birlikte modelin ezberlemesini engellemeye çalıştıgımız bir fonksiyondu ve bu proje için 0.2 ile 0.1 arasında bir değer girmeyi tercih ettim çünkü değişken sayısı az yüksek değer girme durumunda aptallaşabilir modelimiz

2026-02-14 18:12:44.681361: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2
2026-02-14 18:12:44.681392: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2026-02-14 18:12:44.681399: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2026-02-14 18:12:44.681419: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2026-02-14 18:12:44.681429: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
/opt/anaconda3/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, pref

In [9]:
# ikinci bir lstm katmanı ekliyoruz bu katmanı da diğerleri gibi 50 nöronlu yapalım 
model.add(LSTM(units=50,return_sequences=False))
model.add(Dropout(0.2))

In [10]:
#Çıkış katmanını inşa etmemiz gerekiyor
model.add(Dense(units=1)) # Dense çıkış katmanıdır

In [11]:
model.compile(optimizer='adam', loss='mean_squared_error')
# compile modelin nasıl öğreneceğini belirteceğimiz aşamadır 
# optimizer modelin hatayı azaltmak için ağırlıkları nasıl güncelleyeceğini belirleyen algoritmadır
# optimizer olarak ben genelde adam(adaptive moment estimation) ı kullanıyorum

In [12]:
X = np.array(X).astype('float32')
y = np.array(y).astype('float32')
model.fit(X,y, epochs=15, batch_size=32)
#modeli eğitiyoruz
#model eğitiminde genelde oldgu gibi grupları 32 şerli olarak tervih ediyoruz 

Epoch 1/15


/opt/anaconda3/lib/python3.12/site-packages/tensorflow/python/data/ops/structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


7811/7811 ━━━━━━━━━━━━━━━━━━━━ 510s 65ms/step - loss: 0.0011
Epoch 2/15
7811/7811 ━━━━━━━━━━━━━━━━━━━━ 507s 65ms/step - loss: 3.0738e-04
Epoch 3/15
7811/7811 ━━━━━━━━━━━━━━━━━━━━ 565s 72ms/step - loss: 2.9316e-04
Epoch 4/15
7811/7811 ━━━━━━━━━━━━━━━━━━━━ 509s 65ms/step - loss: 2.8715e-04
Epoch 5/15
7811/7811 ━━━━━━━━━━━━━━━━━━━━ 638s 82ms/step - loss: 2.8275e-04
Epoch 6/15
7811/7811 ━━━━━━━━━━━━━━━━━━━━ 509s 65ms/step - loss: 2.7750e-04
Epoch 7/15
7811/7811 ━━━━━━━━━━━━━━━━━━━━ 509s 65ms/step - loss: 2.7283e-04
Epoch 8/15
7811/7811 ━━━━━━━━━━━━━━━━━━━━ 509s 65ms/step - loss: 2.6938e-04
Epoch 9/15
7811/7811 ━━━━━━━━━━━━━━━━━━━━ 513s 66ms/step - loss: 2.6690e-04
Epoch 10/15
7811/7811 ━━━━━━━━━━━━━━━━━━━━ 511s 65ms/step - loss: 2.6449e-04
Epoch 11/15
7811/7811 ━━━━━━━━━━━━━━━━━━━━ 774s 99ms/step - loss: 2.6279e-04
Epoch 12/15
7811/7811 ━━━━━━━━━━━━━━━━━━━━ 526s 67ms/step - loss: 2.5976e-04
Epoch 13/15
7811/7811 ━━━━━━━━━━━━━━━━━━━━ 515s 66ms/step - loss: 2.5937e-04
Epoch 14/15
7811/7811 ━

In [13]:
model.save('live_trackng.h5') 

print("ok")

ok
